In [1]:
from efficientnet import model as efficientnet
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim
from torch.utils.data import Dataset ,DataLoader
import torchvision.transforms as transforms 
import pandas as pd
import numpy as np 
from sklearn.metrics import f1_score, precision_score, recall_score
from matplotlib import pyplot as plt
from PIL import Image
import os,shutil,copy,re,time

In [2]:
#第一次训练模型需要从EfficientNet获取模型，之后不需要，直接读之前训练保存的pkl文件
#现在本地测试，所以把多GPU设置注释掉。之后要设置：batch_size=128,num_workers=8,.from_name("efficientnet-b7")
CLASSNUMBER=53
model=efficientnet.EfficientNet.from_pretrained("efficientnet-b8")
model._fc.out_features=CLASSNUMBER
print(model._fc.in_features)
print("分类数量",model._fc.out_features)
#把模型放到GPU上
model=model.cuda()
#使用多GPU
# model=nn.DataParallel(model)

1280
分类数量 53


In [3]:
dfAllFilePath=pd.read_csv("./dataset_two_path_afterflip.csv")
imageFilesList=dfAllFilePath["breed"].tolist()
classToIndex={
    x:i for i,x in enumerate(dfAllFilePath["breed"].unique())
}
IndexToClass={
    i:x for i,x in enumerate(dfAllFilePath["breed"].unique())
}
normalize=transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

dsTransforms=transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

In [4]:
#由于数据集不同种类图片数量相差太大，所以训练、验证、测试集的划分要按照不同品种来划分
"""
首先把53个不同品种的所有路径、品种分别保存到53个DataFrame中。然后53个DataFrame放到1个List中。
由于之前统计过不同种类的图片数量，所以直接用数量来对大DataFrame切片
"""

breedsList=list(IndexToClass.values())
dfBreedsNumber=pd.read_csv("./cat_breeds_two_total_afterflip.csv")
dfBreedsDict={}
begin=0
end=0
for i in range(len(dfBreedsNumber)):
    breedName=dfBreedsNumber.iat[i,0]
    end=dfBreedsNumber.iat[i,2]+end
    tempDf=dfAllFilePath[begin:end]
    dfBreedsDict[breedName]=tempDf
    begin=end

# 用于存储53个品种的文件地址链接的字典已经创建完成，接下来就分别划分训练、验证、测试集，然后添加到分别添加到大的训练、验证、测试集中
dfTrain=pd.DataFrame(columns=["path","breed"])
dfValidate=dfTrain.copy()
dfTest=dfTrain.copy()
for ele in list(dfBreedsDict.keys()):
    #ele是DataFrame
    tempDf=dfBreedsDict[ele]
    tempTrain,tempValidate,tempTest=np.split(
        tempDf.sample(frac=1,random_state=42),
        [
            int(0.6*len(tempDf)),
            int(0.8*len(tempDf)),
        ]
    )
    dfTrain=dfTrain.append(tempTrain,ignore_index=True)
    dfValidate=dfValidate.append(tempValidate,ignore_index=True)
    dfTest=dfTest.append(tempTest,ignore_index=True)

#创建Dataset对象
class CatDataset(Dataset):
    def __init__(self,df,transform=None):
        self.df=df
        self.transform=transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        imgPath=self.df.iat[idx,0]
        image=Image.open(imgPath)
        if image.mode !="RGB":
            image=image.convert("RGB")
        if self.transform:
            image=self.transform(image)
        breedIndex=classToIndex[self.df.iat[idx,1]]
        return [image,breedIndex]

#创建训练、验证、测试集的Dataset
dsTrain=CatDataset(dfTrain,transform=dsTransforms)
dsValidate=CatDataset(dfValidate,transform=dsTransforms)
dsTest=CatDataset(dfTest,transform=dsTransforms)
#创建训练、验证、测试集的Dataloader
dlTrain=DataLoader(
    dsTrain,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    drop_last=True
)
dlValidate=DataLoader(
    dsValidate,
    batch_size=4,
    shuffle=True,
    num_workers=0,
    drop_last=True
)
dlTest=DataLoader(
    dsTest,
    batch_size=4,
    shuffle=True,
    num_workers=0,
    drop_last=True
)
dataloaders={
    "train":dlTrain,
    "validate":dlValidate,
    "test":dlTest
}
datasets={
    "train":dsTrain,
    "validate":dsValidate,
    "test":dsTest
}

In [5]:
#创建损失计算函数、参数优化器、学习率调度器
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adadelta(model.parameters(),lr=1)
scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode="max",verbose=True)

In [6]:
#定义训练函数，注意，由于要在服务器上训练，所以每次调用训练函数后，加一行保存模型的代码。
def train_model(model,criterion,optimizer,scheduler,epochNumber=42):
    sinceWhole=time.time()
    bestModelState=model.state_dict()
    bestF1=0.0
    bestPrecision=0.0
    bestRecall=0.0
    
    for epoch in range(epochNumber):
        sinceThisEpoch=time.time()
        print("Epoch {}/{}".format(epoch,epochNumber))
        print("-"*20)
        for phase in ["train","validate"]:
            if phase=="train":
                model.train(True)
            else:
                model.train(False)
            print("运行位置打印：训练开始前")

            if phase=="train":
                labelsAll=[]
                predsAll=[]
                i=1
                for data in dataloaders[phase]:
                    print("运行位置打印：训练中",i,end="\t")
                    i+=1
                    inputs,labels=data
                    inputs=inputs.float().cuda()
                    labels=labels.cuda()
                    optimizer.zero_grad()

                    outputs=model(inputs)
                    loss=criterion(outputs,labels)
                    _,preds=torch.max(outputs.data,dim=1)
                    labelsAll.extend(labels.cpu().numpy().tolist())
                    predsAll.extend(preds.cpu().numpy().tolist())

                    loss.backward()
                    optimizer.step()
                
                print("运行位置打印：训练后")
                f1Train=f1_score(labelsAll,predsAll,average="macro")
                precsionTrain=precision_score(labelsAll,predsAll,average="macro")
                recallTrain=recall_score(labelsAll,predsAll,average="macro")

            if phase=="validate":
                labelsAll=[]
                predsAll=[]
                with torch.no_grad():
                    i=0
                    for data in dataloaders[phase]:
                        print("运行位置打印：验证中",i,end="\t")
                        i+=1
                        inputs,labels=data
                        inputs=inputs.float().cuda()
                        labels=labels.cuda()

                        outputs=model(inputs)
                        _,preds=torch.max(outputs.data,dim=1)
                        labelsAll.extend(labels.cpu().numpy().tolist())
                        predsAll.extend(preds.cpu().numpy().tolist())
                print("运行位置打印：验证后")
                f1Validate=f1_score(labelsAll,predsAll,average="macro")
                precsionValidate=precision_score(labelsAll,predsAll,average="macro")
                recallValidate=recall_score(labelsAll,predsAll,average="macro")
                stepSign=f1Validate

                scheduler.step(stepSign)
                if f1Validate>bestF1:
                    bestF1=f1Validate
                    bestPrecision=precsionValidate
                    bestRecall=recallValidate
                    bestModelState=model.state_dict()


        #一个Epoch完成后
        timeUsedThisEpoch=time.time()-sinceThisEpoch
        print("Train - F1: {:.9f}\tprecision: {:.9f}\trecall: {:.9f}\nValidation - F1: {:.9f}\tprecision: {:.9f}\trecall: {:.9f}\nin {:.0f}m {:.0f}s".format(
            f1Train,
            precsionTrain,
            recallTrain,
            f1Validate,
            precsionValidate,
            recallValidate,
            timeUsedThisEpoch//60,timeUsedThisEpoch%60
        ))
        print()

    #训练、验证都完成后
    timeWhole=time.time()-sinceWhole
    print("Training complete in {:.0f}m {:.0f}s".format(timeWhole//60,timeWhole%60))
    print("Best Validation - F1: {:.9f}\tprecision: {:.9f}\trecall: {:.9f}".format(
        bestF1,
        bestPrecision,
        bestRecall
    ))
    print()
    
    model.load_state_dict(bestModelState)
    return model

In [7]:
model=train_model(model,criterion,optimizer,scheduler,epochNumber=2)

7运行位置打印：训练中 738运行位置打印：训练中 739运行位置打印：训练中 740运行位置打印：训练中 741运行位置打印：训练中 742运行位置打印：训练中 743运行位置打印：训练中 744运行位置打印：训练中 745运行位置打印：训练中 746运行位置打印：训练中 747运行位置打印：训练中 748运行位置打印：训练中 749运行位置打印：训练中 750运行位置打印：训练中 751运行位置打印：训练中 752运行位置打印：训练中 753运行位置打印：训练中 754运行位置打印：训练中 755运行位置打印：训练中 756运行位置打印：训练中 757运行位置打印：训练中 758运行位置打印：训练中 759运行位置打印：训练中 760运行位置打印：训练中 761运行位置打印：训练中 762运行位置打印：训练中 763运行位置打印：训练中 764运行位置打印：训练中 765运行位置打印：训练中 766运行位置打印：训练中 767运行位置打印：训练中 768运行位置打印：训练中 769运行位置打印：训练中 770运行位置打印：训练中 771运行位置打印：训练中 772运行位置打印：训练中 773运行位置打印：训练中 774运行位置打印：训练中 775运行位置打印：训练中 776运行位置打印：训练中 777运行位置打印：训练中 778运行位置打印：训练中 779运行位置打印：训练中 780运行位置打印：训练中 781运行位置打印：训练中 782运行位置打印：训练中 783运行位置打印：训练中 784运行位置打印：训练中 785运行位置打印：训练中 786运行位置打印：训练中 787运行位置打印：训练中 788运行位置打印：训练中 789运行位置打印：训练中 790运行位置打印：训练中 791运行位置打印：训练中 792运行位置打印：训练中 793运行位置打印：训练中 794运行位置打印：训练中 795运行位置打印：训练中 796运行位置打印：训练中 797运行位置打印：训练中 798运行位置打印：训练中 799运行位置打印：训练中 800运行位置打印：训练中 801运行位置打印：训练中 802运行位置打印：训练中 803运行位置打印：训练中 804运行位置打印：训练中 805运行位置打印：训练中 806运行位置打印：训练中 807运行位置打印：训练中 808运行位置打

NameError: name 'labelsForF1' is not defined

In [8]:
list1=[1,2,3]
list2=[4,5,6]
list1.extend(list2)
print(list1)

[1, 2, 3, 4, 5, 6]


In [33]:
#计算F1值
targetNumber=torch.zeros((1,CLASSNUMBER))
predictNumber=torch.zeros((1,CLASSNUMBER))
accNUmber=torch.zeros((1,CLASSNUMBER))


In [9]:
#为了使用sklearn库，测试tensor转List
tempTensor=torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
tempTensor.cuda()
print(type(tempTensor))
print(tempTensor.numpy().tolist())


<class 'torch.Tensor'>
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [28]:
temp=dfAllFilePath[0:1]
print(temp)
tempList=[]
tempList.append(temp)
print(type(tempList[0]))
print(tempList[0])

path       breed
0  ./images/cat_breeds_two/Abyssinian/12136161_25...  Abyssinian
<class 'pandas.core.frame.DataFrame'>
                                                path       breed
0  ./images/cat_breeds_two/Abyssinian/12136161_25...  Abyssinian
